In [102]:
import numpy as np
import torch
import pandas as pd
import scipy.stats as stats

pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 10)
np.set_printoptions(linewidth=400)



In [74]:
# Load the data
ratings = pd.read_csv('../../ml-latest-small/ratings.csv')


In [75]:
print(ratings.shape)

(100836, 4)


In [76]:
print(ratings.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [77]:
r = ratings.pivot(index='movieId', columns='userId', values='rating')
print(r.head())

userId   1    2    3    4    5    6    7    8    9    10   ...  601  602  603  604  605  606  607  608  609  610
movieId                                                    ...                                                  
1        4.0  NaN  NaN  NaN  4.0  NaN  4.5  NaN  NaN  NaN  ...  4.0  NaN  4.0  3.0  4.0  2.5  4.0  2.5  3.0  5.0
2        NaN  NaN  NaN  NaN  NaN  4.0  NaN  4.0  NaN  NaN  ...  NaN  4.0  NaN  5.0  3.5  NaN  NaN  2.0  NaN  NaN
3        4.0  NaN  NaN  NaN  NaN  5.0  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  NaN  NaN
4        NaN  NaN  NaN  NaN  NaN  3.0  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
5        NaN  NaN  NaN  NaN  NaN  5.0  NaN  NaN  NaN  NaN  ...  NaN  NaN  NaN  3.0  NaN  NaN  NaN  NaN  NaN  NaN

[5 rows x 610 columns]


In [78]:
r_np = r.to_numpy()
print(r_np.shape)

(9724, 610)


In [79]:
samp_cov = np.cov(r_np, rowvar=False)
print(samp_cov)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [80]:
# Covariance matrix with pandas:
R = (r - r.mean(axis=0, skipna=True)).div(r.std(axis=0, ddof=0, skipna=True) + 1e-8, axis=1)
R = R.fillna(0)
R_np = R.to_numpy()
n_items = R_np.shape[0]
C = (R_np.T @ R_np) / n_items
print(C)

[[ 2.38584938e-02  1.06665081e-05  5.40535573e-06 ...  3.39668346e-03 -2.44988167e-04  6.17897908e-04]
 [ 1.06665081e-05  2.98231173e-03  0.00000000e+00 ... -9.58000517e-05 -2.02425072e-04  4.99557847e-04]
 [ 5.40535573e-06  0.00000000e+00  4.01069515e-03 ... -2.40686929e-04  0.00000000e+00  4.53042362e-04]
 ...
 [ 3.39668346e-03 -9.58000517e-05 -2.40686929e-04 ...  8.54586574e-02  9.14510034e-04  5.82491846e-03]
 [-2.44988167e-04 -2.02425072e-04  0.00000000e+00 ...  9.14510034e-04  3.80501834e-03 -2.81499434e-04]
 [ 6.17897908e-04  4.99557847e-04  4.53042362e-04 ...  5.82491846e-03 -2.81499434e-04  1.33895513e-01]]


In [81]:
def nanvar(
        tensor: torch.Tensor,
        dim: int=None,
        keepdim: bool=False,
        correction: int=1
) -> torch.Tensor:
    count = (~torch.isnan(tensor)).sum(dim=dim, keepdim=keepdim)

    mean = torch.nanmean(tensor, dim=dim, keepdim=True)

    sq_diff = (tensor - mean).pow(2)
    sq_diff = torch.where(
        torch.isnan(sq_diff),
        torch.zeros_like(sq_diff),
        sq_diff)

    sum_sq_diff = sq_diff.sum(dim=dim, keepdim=keepdim)

    divisor = (count - correction).clamp(min=1)

    return sum_sq_diff / divisor.to(sum_sq_diff.dtype)

def nanstd(
        tensor: torch.Tensor,
        dim: int=None,
        keepdim: bool=False,
        correction: int=1
) -> torch.Tensor:

    return nanvar(tensor, dim=dim, keepdim=keepdim, correction=correction).sqrt()

In [94]:
def user_user_covariance_torch(
        r: torch.Tensor
) -> torch.Tensor:

    # r : (|I| x |U|)
    means = torch.nanmean(r, dim=0)
    stds = nanstd(r, dim=0, correction=0)

    R_z = (r - means) / (stds + 1e-8)
    R_z = torch.nan_to_num(R_z, nan=0.0)

    n_items = R_z.shape[0]
    C = (R_z.t() @ R_z) / n_items
    return C

In [95]:
r_torch = torch.tensor(r_np, dtype=torch.float64)
C_torch = user_user_covariance_torch(r_torch)
print(C_torch)

tensor([[ 2.3858e-02,  1.0667e-05,  5.4054e-06,  ...,  3.3967e-03,
         -2.4499e-04,  6.1790e-04],
        [ 1.0667e-05,  2.9823e-03,  0.0000e+00,  ..., -9.5800e-05,
         -2.0243e-04,  4.9956e-04],
        [ 5.4054e-06,  0.0000e+00,  4.0107e-03,  ..., -2.4069e-04,
          0.0000e+00,  4.5304e-04],
        ...,
        [ 3.3967e-03, -9.5800e-05, -2.4069e-04,  ...,  8.5459e-02,
          9.1451e-04,  5.8249e-03],
        [-2.4499e-04, -2.0243e-04,  0.0000e+00,  ...,  9.1451e-04,
          3.8050e-03, -2.8150e-04],
        [ 6.1790e-04,  4.9956e-04,  4.5304e-04,  ...,  5.8249e-03,
         -2.8150e-04,  1.3390e-01]], dtype=torch.float64)


In [84]:
cov_builtin_torch = torch.cov(r_torch.t())
print(cov_builtin_torch.shape)
print(cov_builtin_torch)

torch.Size([610, 610])
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], dtype=torch.float64)


In [85]:
print((C_torch - cov_builtin_torch).abs().max())

tensor(nan, dtype=torch.float64)


In [103]:
R_sci = stats.zscore(r, axis=0, ddof=0, nan_policy='omit')
print(R_sci)
print(R_np)

userId        1    2    3    4         5    ...       606       607       608       609       610
movieId                                     ...                                                  
1       -0.458937  NaN  NaN  NaN  0.371391  ... -1.599069  0.222106 -0.587955 -0.608581  1.530107
2             NaN  NaN  NaN  NaN       NaN  ...       NaN       NaN -1.051514       NaN       NaN
3       -0.458937  NaN  NaN  NaN       NaN  ...       NaN       NaN -1.051514       NaN       NaN
4             NaN  NaN  NaN  NaN       NaN  ...       NaN       NaN       NaN       NaN       NaN
5             NaN  NaN  NaN  NaN       NaN  ...       NaN       NaN       NaN       NaN       NaN
...           ...  ...  ...  ...       ...  ...       ...       ...       ...       ...       ...
193581        NaN  NaN  NaN  NaN       NaN  ...       NaN       NaN       NaN       NaN       NaN
193583        NaN  NaN  NaN  NaN       NaN  ...       NaN       NaN       NaN       NaN       NaN
193585        NaN  N

In [104]:
def sparsify_covariance(
        C: torch.Tensor,
        cov_type: str,
        thr: float=0.0,
        p: float=0.1,
        sparse_tensor: bool=False):
    if cov_type == "standard":
        C_sparse = C
    elif cov_type == "RCV":
        # Generate probability values
        sigma = min((1-p)/3, p/3)
        lim_prob = np.linspace(0,1,1000)
        distr_prob = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-0.5 * ((lim_prob-p)/sigma)**2)
        distr_prob = distr_prob / distr_prob.sum()
        prob_values = np.random.choice(lim_prob, p=distr_prob, size=C.shape[0] ** 2)
        prob_values = torch.FloatTensor(np.sort(prob_values))

        # Assign probability values
        sorted_idx = torch.argsort(C.abs().flatten())
        prob = torch.zeros([C.shape[0] ** 2,]).float().scatter_(0, sorted_idx, prob_values)
        prob = prob.reshape(C.shape)
        prob[torch.eye(prob.shape[0]).long()] = 1 # no removal on the diagonal

        # Drop edges symmetrically
        mask = torch.rand(C.shape) <= prob
        triu = torch.triu(torch.ones(C.shape), diagonal=0).bool()
        mask = mask * triu + mask.t() * ~triu # make resulting matrix symmetric
        C_sparse = torch.where(mask, C, 0)

    elif cov_type == "ACV":
        prob = C.abs() / C.abs().max()
        prob[torch.eye(prob.shape[0]).long()] = 1 # no removal on the diagonal
        mask = torch.rand(C.shape) <= prob
        triu = torch.triu(torch.ones(C.shape), diagonal=0).bool()
        mask = mask * triu + mask.t() * ~triu # make resulting matrix symmetric
        C_sparse = torch.where(mask, C, 0)

    elif cov_type == "hard_thr":
        C_sparse = torch.where(C.abs() > thr, C, 0)
    elif cov_type == "soft_thr":
        C_sparse = torch.where(C.abs() > thr, C - (C>0).float()*thr, 0)

    if sparse_tensor:
        return C_sparse.to_sparse()

    return C_sparse